In [ ]:
from datahandler import DataHandler
from models.unet import *
from generator import *
from params import *
from callbacks import getCallbacks

from tqdm import tqdm
import os
import skimage.io as io

from keras.models import *
from keras import backend as K

%matplotlib inline
import matplotlib.pyplot as plt

import argparse
import sys

In [ ]:
#set common variables
epochs = 25
batch_size = 32
verbose = 1


resetSeed()

In [ ]:
#Get data and generators
dh = DataHandler()
tr_images, tr_masks, te_images, te_masks = dh.getData()

In [ ]:
unet_model = getUnet()
unet_graph = tf.get_default_graph()

with unet_graph.as_default():
    unet_model.load_weights('logs/unet/kfold_unet/kfold_unet_dice_DA_K1/kfold_unet_dice_DA_K1_weights.h5')

In [ ]:
import random

def generator(images, masks, batch_size):
    i = 0
    reset = False
    while True:
        batch_features = []
        batch_labels = []
        
        c = batch_size * i 
        
        if reset:
            i = 0
            c = 0
            reset = False

        for j in range(c, c + batch_size):
            if j >= len(images):
                reset = True
                continue
            
            if j == 0:
                res1 = np.expand_dims(np.zeros(images[j].shape), axis=0)
            else:
                img1 = np.expand_dims(images[j-1], axis=0)
                with unet_graph.as_default():
                    res1 = unet_model.predict(img1)

            img2 = np.expand_dims(images[j], axis=0) 
            with unet_graph.as_default():
                res2 = unet_model.predict(img2)

            if j == images.shape[0]-1:
                res3 = np.expand_dims(np.zeros(images[j].shape), axis=0)
            else:
                img3 = np.expand_dims(images[j+1], axis=0)
                with unet_graph.as_default():
                    res3 = unet_model.predict(img3)

            res = np.concatenate((res1,res2,res3), axis=0)
            res[res>=0.5] = 1
            res[res<0.5] = 0
            
            mask = masks[j]
            mask[mask == 255] = 1
            
            batch_features.append(res)
            batch_labels.append(mask)
        i += 1
        
        if len(batch_features) == 0:
            res1 = np.expand_dims(np.zeros(images[0].shape), axis=0)
            res2 = np.expand_dims(np.zeros(images[0].shape), axis=0)
            res3 = np.expand_dims(np.zeros(images[0].shape), axis=0)
            res = np.concatenate((res1,res2,res3), axis=0)
            
            mask = np.zeros(images[0].shape)
            
            batch_features.append(res)
            batch_labels.append(mask)
            

        yield np.array(batch_features), np.array(batch_labels)


In [ ]:
from keras.models import Model
from keras.layers import Input, MaxPooling2D, Dropout, Conv2D, Conv2DTranspose, TimeDistributed, Bidirectional, ConvLSTM2D
from keras import backend as K
import tensorflow as tf
from keras.optimizers import RMSprop, Adam, SGD
from keras.losses import binary_crossentropy
from keras.callbacks import ModelCheckpoint
from losses import *
import math

train_generator = generator(tr_images, tr_masks, 4)
val_generator = generator(te_images, te_masks, 4)

lstm_graph = tf.get_default_graph()

with lstm_graph.as_default():

    inputs = Input((3, 256, 256, 1))

    bclstm = Bidirectional(ConvLSTM2D(32, 3, return_sequences = True, padding='same', activation = 'relu'))(inputs)
    bclstm = Bidirectional(ConvLSTM2D(32, 3, return_sequences = True, padding='same', activation = 'relu'))(bclstm)

    pool = TimeDistributed(MaxPooling2D(pool_size=2))(bclstm)

    bclstm = Bidirectional(ConvLSTM2D(64, 3, return_sequences = True, padding='same', activation = 'relu'))(pool)
    bclstm = Bidirectional(ConvLSTM2D(64, 3, return_sequences = True, padding='same', activation = 'relu'))(bclstm)
    bclstm = Bidirectional(ConvLSTM2D(64, 3, padding='same', activation = 'relu'))(bclstm)

    up = Conv2DTranspose(64,3, strides=2, padding='same', activation = 'relu')(bclstm)
    conv = Conv2D(64, 3, activation = 'relu', padding='same')(up)

    outputs = Conv2D(1, (1,1), activation = 'sigmoid')(conv)

    model = Model(input = inputs, output = outputs)
    
    model.compile(optimizer = Adam(lr = 1e-4),
            loss = binary_crossentropy, metrics = [dice_coef])
    
    model.fit_generator(train_generator, validation_data=val_generator, 
                        validation_steps=math.floor(len(te_images)/4),
                        callbacks = [ModelCheckpoint('logs/LSTM/', monitor='val_loss', verbose=1, save_best_only=True)],
                        steps_per_epoch=math.floor(len(te_images)/4), epochs=30)

In [ ]:
j = 1925
with unet_graph.as_default():
    img1 = np.expand_dims(tr_images[j-1], axis=0)
    res1 = unet_model.predict(img1)
    res1[res1>=0.5] = 1
    res1[res1<0.5] = 0
    
    img2 = np.expand_dims(tr_images[j], axis=0) 
    res2 = unet_model.predict(img2)
    res2[res2>=0.5] = 1
    res2[res2<0.5] = 0
    
    img3 = np.expand_dims(tr_images[j+1], axis=0)
    res3 = unet_model.predict(img3)
    res3[res3>=0.5] = 1
    res3[res3<0.5] = 0
    
plt.gray()

res = np.array([np.concatenate((res1,res2,res3), axis=0)])
res[res >= 0.5] = 1
res[res < 0.5] = 0

print(res.shape)

with lstm_graph.as_default():
    r = model.predict(res)
    r[r<0.5] = 0
    r[r>0.5] = 1
    print(r.shape)

In [ ]:

plt.imshow(np.squeeze(tr_images[j] + res2[0] * 255 ))

In [ ]:
plt.imshow(np.squeeze(tr_images[j] + r[0] * 255 ))

In [ ]:
plt.imshow(np.squeeze(tr_images[j] + tr_masks[j]))

In [ ]:
from sklearn.metrics import f1_score
mask = tr_masks[j]
mask[mask==255] = 1

res2[res2>=0.5] = 1
res2[res2<0.5] = 0

print(f1_score(r[0].flatten(), mask.flatten()))
print(f1_score(res2[0].flatten(), mask.flatten()))      